In [1]:
#packages

import nltk
import re
from genderize import Genderize
from nltk.corpus import stopwords
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import pickle
import pandas as pd

#Variables(for now):
name_probability_list = {}


In [2]:
#Ingestion

#filejson = "C:/Users/danie/Desktop/bbc_news_list_uk.json"
filecsv = r"C:\Users\danie\Documents\GitHub\Masters-Thesis\bbc_news_list_uk.csv"
#filecsv = r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/bbc_news_list_uk.csv"
article_df = pd.read_csv(filecsv)
article_df = article_df.assign(Article_Number=range(len(article_df)))
article_df = article_df.reset_index()
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15825 entries, 0 to 15824
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              15825 non-null  int64 
 1   tags               1 non-null      object
 2   title              15825 non-null  object
 3   news_post_date     15825 non-null  object
 4   raw_content        15468 non-null  object
 5   content            15468 non-null  object
 6   url                15825 non-null  object
 7   author             1453 non-null   object
 8   language           15825 non-null  object
 9   id                 15825 non-null  object
 10  region             15488 non-null  object
 11  short_description  15825 non-null  object
 12  category           15825 non-null  object
 13  crawled_at         15825 non-null  object
 14  Article_Number     15825 non-null  int32 
dtypes: int32(1), int64(1), object(13)
memory usage: 1.8+ MB


In [3]:
year = article_df['news_post_date'].str[:4]
article_df['year']=year

In [4]:
# tokenize sentences in an article 

def split_sentences(article, article_id, year):
    pattern = r'(?<=[a-z0-9"]) *[.?!] *(?=[A-Z])'
    article = re.sub(pattern, r'\g<0> ', article)
    sentences = nltk.sent_tokenize(article)
    sentences_with_id = [(sentence, article_id, year) for sentence in sentences]
    return sentences_with_id

sentences_list = []

# add sentences to a new DF along with article ID 
for article, article_id, year in article_df[['content','Article_Number', 'year']].values:
    sentences = split_sentences(str(article), article_id, year)
    sentences_list.extend(sentences)

sentences_df = pd.DataFrame(sentences_list, columns= ['sentences', 'article_id', 'year'])

In [17]:
def PreProcessing (sentence):
    Male_count = 0
    Female_count = 0
    APIcallfail= 0

#regex_cleanup
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub("\d+", " ", sentence)
    sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)

#tokenize
    sentence =  nltk.TweetTokenizer().tokenize(sentence)

#tag_and_stem
    tagged_sentence = nltk.tag.pos_tag(sentence)
    lemma = nltk.stem.WordNetLemmatizer()
    pn_tags = {'NNP', 'NNPS'}

    new_words = []
    proper_nouns = []

    for word, tag in tagged_sentence: 
        if tag not in pn_tags: 
            if tag.startswith("V"):
                lemmas = lemma.lemmatize(word, "v")
            else: 
                lemmas = lemma.lemmatize(word)
            new_words.append((lemmas))
        else:
            proper_nouns.append([word, tag])
            #i think the namesshould be dropped.
    sentence = new_words

#name_gender
    #nltk_results = ne_chunk(tagged_sentence)
    nltk_results = ne_chunk(proper_nouns)

    for nltk_result in nltk_results:
        if type(nltk_result) == Tree:
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            if nltk_result.label() == "PERSON":
                name = name.split(' ')[0]
                try: 
                    word_gender = name_probability_list.get(name)
                    if word_gender is None:
                        word_gender = Genderize().get1(name).get('gender')
                        name_probability_list[name] = word_gender
                    if word_gender == 'male':
                        Male_count += 1
                    if word_gender== 'female':
                        Female_count += 1
                except Exception as exception:
                    APIcallfail +=1

#contractions
    new_text = []
    for word in new_words:
        contraction = contractions.get(word)
        if contraction is None:
            new_text.append(word)
        else:
            for word in contraction.split():
                new_text.append(word)
    sentence = new_text

#gendered_count
    for w in sentence:
        if w in male_list:
            Male_count += 1
        if w in female_list:
            Female_count += 1

#remove_stopwords
    stops = set(stopwords.words("english"))
    sentence = [w for w in sentence if not w in stops]

#remove_leakage
    new_sent = [x for x in sentence if x not in male_list]
    new_sent = [x for x in new_sent if x not in female_list]
    sentence = new_sent

    sentence = [x.lower() for x in sentence]

    print(sentence)
    return sentence, Male_count, Female_count, APIcallfail



In [24]:
sample_df = sentences_df.iloc[0:100]
sample_df['encoded_sentences'] = sample_df['sentences'].apply(PreProcessing)

#sentences_df['encoded_sentences'] = sentences_df['sentences'].apply(PreProcessing)

['the', 'heroin', 'substitute', 'methadone', 'use', 'way', 'wean', 'addict', 'heroin', 'substitute', 'also', 'become', 'addictive']
['earlier', 'year', 'debate', 'break', 'director', 'say', 'effort', 'need', 'get', 'people', 'drug', 'include', 'methadone', 'abstinence']
['but', 'group', 'specialist', 'include', 'university', 'professor', 'doctor', 'treat', 'addict', 'respond', 'if', 'policy', 'maker', 'heed', 'critic', "'", 'advice', 'close', 'methadone', 'treatment', 'impose', 'arbitrary', 'time', 'limit', 'administration', 'community', 'anticipate', 'overdose', 'death', 'crime']
['so', 'recover', 'addict', 'think']
['use', 'methadone', 'five', 'year', 'help', 'wean', 'heroin', 'felt', 'quit', 'drug', 'substitute', 'last', 'i', 'think', 'get', 'get', 'grip', 'prison', 'since', 'i', 'half', 'life', 'tell', 'the']
['inmate', 'live', 'specialist', 'help', 'prisoner', 'achieve', 'abstinence']
['i', 'hear', 'give', 'wake', 'call', 'i', 'need', 'say']
['prescribe', 'methadone', 'synthetic',

<ipython-input-24-4a9ccb6cffaa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['encoded_sentences'] = sample_df['sentences'].apply(PreProcessing)


In [28]:
sample_df_split = pd.DataFrame(sample_df["encoded_sentences"].to_list(), columns=['pre_processed_sent','male_count','female_count','apicall_fail'])
#result = pd.concat([sample_df, sample_df_split], ignore_index=True, sort=False)
result = pd.concat([sample_df, sample_df_split.reindex(sample_df.index)], axis=1)

In [41]:
#save the api calls!!!
with open('gender_api_calls.pickle', 'wb') as handle:
    pickle.dump(name_probability_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
#REFERENCE LISTS--TO BE CHANGED
male_list= {"man", "men", "mister", "he", "him", "Mr.", "he", "his","he's", "hes", "father", "dad", "daddy", "grandpa", "grandfather", "husband"}
female_list ={"woman", "women", "missus", "misses", "Ms.", "Mrs.", "her", "she", "hers", "mother", "mom", "mommy", "aunt", "grandmother", "grandma", "wife", "wive"}

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
def compare_count(male_col, female_col): 
    """This function compares the count of female to male pronouns. It will output "1" if male count bigger
    than female count, "neutral" if the count is equal, and "female" if there is a higher female count. 
    The function returns strings because we need categorical variables for log reg to run"""
    if female_col > male_col:
        return "0"
    elif male_col > female_col:
        return "1"
    else: 
        return None
sentences_df['col_type'] = sentences_df.apply(lambda row: compare_count(row['male_count'], row['female_count']),axis=1)
sentences_df

In [ ]:
#remove columns with "None" in the col_type 
sentences_df = sentences_df[sentences_df["col_type"].notnull()]
#sentences_df